# Tutorial for CLI tool `clkhash`

For this tutorial we are going to process a data set for private linkage with clkhash using the command line tool `clkutil`. Note you can also use the Python API.

The Python package `recordlinkage` has a [tutorial](http://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html) linking data sets in the clear, we will try duplicate that in a privacy preserving setting.

First install clkhash, recordlinkage and a few data science tools (pandas and numpy).

In [ ]:
!pip install -U clkhash recordlinkage numpy pandas

In [1]:
import io
import json
import numpy as np
import pandas as pd

In [2]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

## Data Exploration

First we have a look at the dataset.

In [3]:
dfA, dfB = load_febrl4()

dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


For this linkage we will **not** use the social security id column.

In [4]:
dfA.columns

Index(['given_name', 'surname', 'street_number', 'address_1', 'address_2',
       'suburb', 'postcode', 'state', 'date_of_birth', 'soc_sec_id'],
      dtype='object')

In [5]:
dfA.to_csv('PII_a.csv')

## Hashing Schema Definition

A hashing schema instructs clkhash how to treat each column for generating CLKs. A detailed description of the hashing schema can be found in the [api docs](http://clkhash.readthedocs.io/en/latest/schema.html). We will ignore the columns 'rec_id' and 'soc_sec_id' for CLK generation.



In [6]:
%%writefile schema.json
{
  "version": 1,
  "clkConfig": {
    "l": 1024,
    "k": 30,
    "hash": {
      "type": "doubleHash"
    },
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
        "info": "",
        "salt": "",
        "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "rec_id",
      "ignored": true
    },
    {
      "identifier": "given_name",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "surname",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "street_number",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "weight": 1 }
    },
    {
      "identifier": "address_1",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "address_2",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "suburb",
      "format": { "type": "string", "encoding": "utf-8" },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "postcode",
      "format": { "type": "integer", "minimum": 1000, "maximum": 9999 },
      "hashing": { "ngram": 1, "positional": true, "weight": 1 }
    },
    {
      "identifier": "state",
      "format": { "type": "string", "encoding": "utf-8", "maxLength": 3 },
      "hashing": { "ngram": 2, "weight": 1 }
    },
    {
      "identifier": "date_of_birth",
      "format": { "type": "integer" },
      "hashing": { "ngram": 1, "positional": true, "weight": 1 }
    },
    {
      "identifier": "soc_sec_id",
      "ignored": true
    }
  ]
}

Writing schema.json


## Hash the data

We can now hash our PII data from the CSV file using our defined schema. We must provide two *secret keys* to this command - these keys have to be used by both parties hashing data. For this toy example we will use the keys _'key1'_ and _'key2'_, for real data, make sure that the keys contain enough entropy, as knowledge of these keys is sufficient to reconstruct the PII information from a CLK! Also, **do not share these keys with anyone, except the other participating party.**

In [7]:
!clkutil hash PII_a.csv key1 key2 schema.json clks_a.json --validate false

generating CLKs: 100%|█| 5.00k/5.00k [00:05<00:00, 810clk/s, mean=885, std=33.6]
CLK data written to clks_a.json


## Inspect the output

clkhash has hashed the PII, creating a Cryptographic Longterm Key for each entity. The output of `generate_clk_from_csv` shows that the mean popcount is quite high (885 out of 1024) which can effect accuracy.

There are two ways to control the popcount:
- You can change the _'k'_ value in the clkConfig section of the schema. It controls the number of entries in the CLK for each n-gram
- or you can modify the individual _'weight'_ values for the different fields. It allows to tune the contribution of a column to the CLK. This can be used to de-emphasise columns which are less suitable for linkage (e.g. information that changes frequently).

First, we will change the value of *k* from 30 to 15.

In [8]:
schema = json.load(open('schema.json', 'rt'))
schema['clkConfig']['k'] = 15
json.dump(schema, open('schema.json', 'wt'))

!clkutil hash PII_a.csv key1 key2 schema.json clks_a.json --validate false

generating CLKs: 100%|█| 5.00k/5.00k [00:07<00:00, 671clk/s, mean=647, std=44.3]
CLK data written to clks_a.json


And now we will modify the weights to de-emphasise the contribution of the address related columns.

In [9]:
schema = json.load(open('schema.json', 'rt'))
schema['clkConfig']['k'] = 20
address_features = ['street_number', 'address_1', 'address_2', 'suburb', 'postcode', 'state']
for feature in schema['features']:
    if feature['identifier'] in address_features:
        feature['hashing']['weight'] = 0.5
json.dump(schema, open('schema.json', 'wt'))

!clkutil hash PII_a.csv key1 key2 schema.json clks_a.json --validate false

generating CLKs: 100%|█| 5.00k/5.00k [00:05<00:00, 759clk/s, mean=600, std=39.6]
CLK data written to clks_a.json


Each CLK is serialized in a JSON friendly base64 format:

In [10]:
!jq .clks[0] clks_a.json

"8x7+E1f+z/nOOvfrc6bjlL7sLLeNOyuP6MyT2nM7qbXWI/fpqe3CVlruV/bdOxYoxvdsVXqOLj9ufI4Gy9xk6g5L8SsXv6FihikoO7pf95tk9VEn0GlNxyl//4qe9Hp/vrP+0fuwf/vrSBdfqs500jlO+03N5d9oBrvrj7nT6JA="


## Hash data set B

Now we hash the second dataset using the same keys and same schema.

In [11]:
dfB.to_csv('PII_b.csv')

!clkutil hash PII_b.csv key1 key2 schema.json clks_b.json --validate false

generating CLKs: 100%|█| 5.00k/5.00k [00:05<00:00, 905clk/s, mean=591, std=45.4]
CLK data written to clks_b.json


## Find matches between the two sets of CLKs

We have generated two sets of CLKs which represent entity information in a privacy-preserving way. The more similar two CLKs are, the more likely it is that they represent the same entity.

For this task we will use the entity service, which is provided by Data61. 
The necessary steps are as follows:
- The analyst creates a new project of the output type 'mapping'. He will receive a set of credentials from the server.
- He then distributes the 'update_tokens' between the participating data providers.
- The data providers then individually upload their respective CLKs.
- After the entity service successfully computed the mapping, it can be accessed by providing the 'result_token'

In [12]:
!clkutil status

Connecting to Entity Matching Server: https://es.data61.xyz
Response: 200
Status: ok
{"status": "ok", "number_mappings": 5212, "rate": 3409331}


The analyst creates a new project on the entity service by providing the hashing schema, result type and a threshold.
The server returns a set of credentials which provide access to the further steps for project.

In [13]:
!clkutil create --schema schema.json --output credentials.json --type "mapping" --threshold 0.9

Entity Matching Server: https://es.data61.xyz
Checking server status
Server Status: ok
Schema: {'version': 1, 'clkConfig': {'l': 1024, 'k': 20, 'hash': {'type': 'doubleHash'}, 'kdf': {'type': 'HKDF', 'hash': 'SHA256', 'info': '', 'salt': '', 'keySize': 64}}, 'features': [{'identifier': 'rec_id', 'ignored': True}, {'identifier': 'given_name', 'format': {'type': 'string', 'encoding': 'utf-8'}, 'hashing': {'ngram': 2, 'weight': 1}}, {'identifier': 'surname', 'format': {'type': 'string', 'encoding': 'utf-8'}, 'hashing': {'ngram': 2, 'weight': 1}}, {'identifier': 'street_number', 'format': {'type': 'integer'}, 'hashing': {'ngram': 1, 'positional': True, 'weight': 0.5}}, {'identifier': 'address_1', 'format': {'type': 'string', 'encoding': 'utf-8'}, 'hashing': {'ngram': 2, 'weight': 0.5}}, {'identifier': 'address_2', 'format': {'type': 'string', 'encoding': 'utf-8'}, 'hashing': {'ngram': 2, 'weight': 0.5}}, {'identifier': 'suburb', 'format': {'type': 'string', 'encoding': 'utf-8'}, 'hashing':

The returned credentials contain a 
- 'resource_id', which identifies the project
- 'result_token', which gives access to the mapping result, once computed
- 'upload_tokens', one for each provider, allows uploading CLKs.

In [20]:
credentials = json.load(open('credentials.json', 'rt'))
!python -m json.tool credentials.json

{
    "resource_id": "d49ca9f4791c71c15ca39331248654686c7d20c6b3ab051e",
    "result_token": "f11096415b3d5ec900986c913321026d4f8d4f39469a1441",
    "update_tokens": [
        "da558803b84df961776eb3c4077e9fc3d04cf59fe1d21244",
        "ea7415fb0acb29af75e15f723f5aa195808cafb2db780851"
    ]
}


### Uploading the CLKs to the entity service
Each party individually uploads its respective CLKs to the entity service. They need to provide the `resource_id`, which identifies the correct mapping, and an `update_token`.

In [15]:
!clkutil upload \
       --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['update_tokens'][0]}" \
        --output "upload_a.json" \
       "clks_a.json"
    
!clkutil upload \
       --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['update_tokens'][1]}" \
        --output "upload_b.json" \
       "clks_b.json"

Uploading CLK data from clks_a.json
To Entity Matching Server: https://es.data61.xyz
Mapping ID: d49ca9f4791c71c15ca39331248654686c7d20c6b3ab051e
Checking server status
Status: ok
Uploading CLK data to the server
Uploading CLK data from clks_b.json
To Entity Matching Server: https://es.data61.xyz
Mapping ID: d49ca9f4791c71c15ca39331248654686c7d20c6b3ab051e
Checking server status
Status: ok
Uploading CLK data to the server


## Results

Now after some delay (depending on the size) we can fetch the mask.
This can be done with clkutil:

In [16]:
!clkutil results \
        --mapping="{credentials['resource_id']}" \
        --apikey="{credentials['result_token']}" --output results.txt
with open('results.txt') as f:
    str_mapping = eval(f.readline())['mapping']
    mapping = {int(k): int(v) for k,v in str_mapping.items()}
print('The service linked {} entities.'.format(len(mapping)))

Checking server status
Status: ok
Response code: 200
Received result
The service linked 3666 entities.


Let's investigate some of those matches and the overall matching quality

In [17]:
with open('PII_a.csv', 'rt') as f:
    a_raw = f.readlines()
with open('PII_b.csv', 'rt') as f:
    b_raw = f.readlines()
print('idx_a, idx_b, rec_id_a, rec_id_b')
print('--------------------------------')
for a_i in range(10):
    if a_i in mapping:
        a_data = a_raw[a_i + 1].split(',')
        b_data = b_raw[mapping[a_i] + 1].split(',')
        print('{}, {}, {}, {}'.format(a_i+1, mapping[a_i]+1, a_data[0], b_data[0]))

TP = 0; FP = 0
for a_i in mapping:
    if a_raw[a_i + 1].split(',')[0].split('-')[1] == b_raw[mapping[a_i] + 1].split(',')[0].split('-')[1]:
        TP += 1
    else:
        FP += 1
FN = len(a_raw)-1 - len(mapping)  # we know that both datasets contains the same entities.
TN = 0
print('--------------------------------')
print('Precision: {}, Recall: {}, Accuracy: {}'.format(TP/(TP+FP), TP/(TP+FN), (TP+TN)/(TP+TN+FP+FN)))

idx_a, idx_b, rec_id_a, rec_id_b
--------------------------------
2, 1583, rec-1016-org, rec-858-dup-0
3, 4679, rec-4405-org, rec-1677-dup-0
4, 3206, rec-1288-org, rec-4181-dup-0
5, 4742, rec-3585-org, rec-642-dup-0
6, 1742, rec-298-org, rec-4331-dup-0
7, 2923, rec-1985-org, rec-673-dup-0
9, 2890, rec-1473-org, rec-1352-dup-0
10, 2395, rec-453-org, rec-3225-dup-0
--------------------------------
Precision: 0.001363884342607747, Recall: 0.0037341299477221808, Accuracy: 0.001


Precision tells us about how many of the found matches are actual matches. The score of 1.0 means that we did perfectly in this respect, however, recall, the measure of how many of the actual matches were correctly identified, is quite low with only 73%.

Let's go back and create another mapping with a `threshold` value of `0.8`.

Great, for this threshold value we get a precision of 100% and a recall of 95.3%. 

The explanation is that when the information about an entity differs slightly in the two datasets (e.g. spelling errors, abbrevations, missing values, ...) then the corresponding CLKs will differ in some number of bits as well. For the datasets in this tutorial the perturbations are such that only 72.7% of the derived CLK pairs overlap more than 90%. Whereas almost all matching pairs overlap more than 80%.

If we keep reducing the threshold value, then we will observe that both precision and recall will decrease. Thus it is important to choose an appropriate threshold for the amount of perturbations present in the data.

This concludes the tutorial. Feel free to go back to the CLK generation and experiment on how different setting will affect the matching quality.